# Libs

In [1]:
# 기본 데이터 처리 라이브러리
import os
import sys
import pandas as pd
pd.set_option('display.max_columns',None)
import numpy as np
from tqdm import tqdm

# 지리 정보 처리 관련 라이브러리
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString, box

# 머신러닝 관련 라이브러리
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from catboost import CatBoostClassifier

# 시각화 관련 라이브러리
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm, rc
font_path = '/usr/local/src/python3.10/lib/python3.10/site-packages/matplotlib/mpl-data/fonts/ttf/KoPub Dotum Medium.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

# DuckDB 데이터베이스 연결
import duckdb

# Data Import
- SPLT_LK_LT 는 링크 길이
- ARI Data에서 DGDGR_BASIC 컬럼이 ARI 산출값
- 설치 전 후의 ARI 값을 위해서는 ACC_NO를 활용, 전후 기준으로 ARI 재산출이 필요할 것으로 보임

In [2]:
link = gpd.read_file('../../KoRoad/TAAS/AN04/세분화링크/세분화링크_20240822.shp')
acc_with_link = gpd.read_file('../../KoRoad/TAAS/AN04/세분화링크/세분화링크_사고결합_20240822.shp')
ari_with_link = pd.read_csv('../../KoRoad/TAAS/AN04/세분화링크/세분화링크_ARI산출.csv')


In [3]:
link.head()

,OBJECTID,LINK_ID,SPLT_LINK,SPLT_LK_LT,SIDO_CD,SGG_CD,BJD_CD,FRST_REG_I,LAST_UPUSR,FRST_REG_D,LAST_UPD_D,SPLT_LK_CD,geometry
0,392964613,1000000301,1000000301_11110126_1,30.000100,11,11110,11110126,TODBC.War,TODBC.War,24/08/22,24/08/22,1,"LINESTRING (954019.581 1952436.263, 954049.581..."
1,392964614,1000000301,1000000301_11110126_2,30.000100,11,11110,11110126,TODBC.War,TODBC.War,24/08/22,24/08/22,1,"LINESTRING (954140.449 1952436.807, 954170.449..."
2,392964615,1000000301,1000000301_11110126_3,90.869315,11,11110,11110126,TODBC.War,TODBC.War,24/08/22,24/08/22,4,"LINESTRING (954049.581 1952436.398, 954140.449..."
3,392964616,1000000302,1000000302_11110126_1,30.000100,11,11110,11110126,TODBC.War,TODBC.War,24/08/22,24/08/22,1,"LINESTRING (954170.473 1952436.942, 954200.472..."
4,392964617,1000000302,1000000302_11110126_2,30.000100,11,11110,11110126,TODBC.War,TODBC.War,24/08/22,24/08/22,1,"LINESTRING (954245.297 1952437.585, 954275.296..."


In [4]:
acc_with_link.head()

,OBJECTID,ACC_NO,LINK_ID,SPLT_LINK,INSTT_CD,ACC_YEAR,UTMK_X,UTMK_Y,EXPWY_AT,MAP_DIST,DTH_CNT,SE_CNT,SL_CNT,WND_CNT,FRST_REG,LAST_UPD,geometry
0,125352465,2020112700100556,2350442300,2350442300_41630114_1,0,2020,964669.0,1981081.0,N,1.369028,0,0,3,0,TODBC.war,22/05/19,POINT (964669 1981081)
1,125352481,2020112700100557,1040033100,1040033100_11215104_2,0,2020,965453.0,1949714.0,N,10.193225,0,0,1,0,TODBC.war,22/05/19,POINT (965453 1949714)
2,125352497,2020112700100557,1040013304,1040013304_11215104_1,0,2020,965453.0,1949714.0,N,4.829617,0,0,1,0,TODBC.war,22/05/19,POINT (965453 1949714)
3,125352513,2020112700100558,1030015601,1030015601_11200109_3,0,2020,958030.0,1950925.0,N,2.073612,0,0,1,0,TODBC.war,22/05/19,POINT (958030 1950925)
4,125352529,2020112700100559,1200023000,1200023000_11620102_8,0,2020,949761.0,1941300.0,N,0.645816,0,0,1,0,TODBC.war,22/05/19,POINT (949761 1941300)


In [5]:
ari_with_link.head()

,SPLT_LINK,STD_YEAR,DTH_ACC_CNT,SE_ACC_CNT,SL_ACC_CNT,WND_ACC_CNT,DTH_DNV_CNT,SE_DNV_CNT,SL_DNV_CNT,WND_DNV_CNT,SPLIT_LINK_LT,DGDGR_BASIC,FRST_REGISTER_ID,FRST_REGIST_DT,LAST_UPDUSR_ID,LAST_UPDT_DT
0,1000000301_11110126_1,2023,0,0,1,0,0,0,1,0,30.000100,0.1133,TODBC.war,2024-08-23 17:21:16,TODBC.war,2024-08-23 17:21:16
1,1000000301_11110126_3,2023,0,0,3,0,0,0,4,0,90.869315,0.4490,TODBC.war,2024-08-23 17:21:16,TODBC.war,2024-08-23 17:21:16
2,1000000302_11110126_1,2023,0,5,0,0,0,5,0,0,30.000100,48.6665,TODBC.war,2024-08-23 17:21:16,TODBC.war,2024-08-23 17:21:16
3,1000000302_11110126_3,2023,0,0,0,1,0,0,0,1,44.826804,0.0368,TODBC.war,2024-08-23 17:21:16,TODBC.war,2024-08-23 17:21:16
4,1000000303_11110126_2,2023,0,0,2,0,0,0,3,0,37.474320,0.5444,TODBC.war,2024-08-23 17:21:16,TODBC.war,2024-08-23 17:21:16
